# Process the data obtained from polarimetry and reorganize the data into the correct folders

In [1]:
%load_ext autoreload
%autoreload 2

import os
from processingmm import batch_processing

 Could not import win32api and/or win32con


In [9]:
# set the parameters to run the script
directories = ['/media/elea/FileBackUP/NPP/600']
calib_directory = '/media/elea/FileBackUP/NPP/calib'
    
# set the parameters to be used for the line visualisation
# NB: parameter file accessible in ./src/processingmm/data/parameters_visualisations.json
parameter_set = 'default'

# set run_all to true in order to run the pipeline on all the folders (even the ones already processed)
run_all = True

# PDDN_mode can be set to:
# 1. 'no': processes without using the PDDN
# 2. 'pddn': processes with PDDN when available (for 550nm and 650nm)
# 3. 'both': processes both with PDDN when available and without PDDN
PDDN_mode = 'no'

# do not specify unless you want to use a custom path for the PDDN models
PDDN_models_path = None

# Set the wavelengths to be processed
# 1. 'all': processes all the available wavelenght
# 2. [xxx, yyy]: processes only the wavelenghts 'xxx' and 'yyy'
wavelengths = [550, 600]

# Processing mode
# 1. 'no_viz': processes only the MM - no visualization at all. useful for fast computation
# 2. 'default': processes the MM and plots the polarimetric parameters maps (i.e. depolarization, azimuth, 
# retardance, diattenuation, azimuth local variability)
# 3. 'full': do like default, and additionally plot the MM components, as well as the line
# visualization

# define if pdf figures should be saved (takes a lot of time) - no impact when processing_mode is set to no_viz

# NB: processing time without PDDN
# 'no_viz': 0.71s
# 'default', save_pdf_figs False: 2.25s
# 'default', save_pdf_figs True: 3.60s
# 'full', save_pdf_figs False: 3.95s
# 'full', save_pdf_figs True: 8.07s
processing_mode = 'default'
save_pdf_figs = False

# define if the wavelenghts should be aligned before processing - and used for the computation
align_wls = False

parameters = batch_processing.get_parameters(directories, calib_directory, wavelengths, parameter_set = parameter_set, 
                                PDDN_mode = PDDN_mode, PDDN_models_path = PDDN_models_path, 
                                processing_mode = processing_mode, run_all = run_all, 
                                save_pdf_figs = save_pdf_figs, align_wls = align_wls)

In [17]:
%%time
times = batch_processing.batch_process_master(parameters)
times

processing without PDDN...


  0%|                                                                                                                                                                                                                    | 0/117 [00:00<?, ?it/s]

Processing: /media/elea/FileBackUP/NPP/600/TU/2023-02-22_T_HORAO-84-AF_FR_S3_1


/home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/MM_processing.py:264: RuntimeWarning: Mean of empty slice
  if math.isnan(np.nanmean(azi)):
  1%|█▋                                                                                                                                                                                                          | 1/117 [00:03<05:53,  3.05s/it]

Processing: /media/elea/FileBackUP/NPP/600/TU/2022-11-28_T_HORAO-68-AF_FR_S1_1


/home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/MM_processing.py:264: RuntimeWarning: Mean of empty slice
  if math.isnan(np.nanmean(azi)):
  2%|███▍                                                                                                                                                                                                        | 2/117 [00:05<04:56,  2.58s/it]

Processing: /media/elea/FileBackUP/NPP/600/TU/2023-09-21_T_HORAO-104-AF_FR_S5_1


/home/elea/Documents/HORAO/repositories/processingMM/src/processingmm/MM_processing.py:264: RuntimeWarning: Mean of empty slice
  if math.isnan(np.nanmean(azi)):
  2%|███▍                                                                                                                                                                                                        | 2/117 [00:07<07:05,  3.70s/it]

KeyboardInterrupt



In [28]:
%%time
from processingmm.addons import visualization_lines

# Approximate time to process the line visualization save_pdf_figs True : 3.85s
# Approximate time to process the line visualization save_pdf_figs False : 4.92s
times = visualization_lines.batch_visualization(parameters)
times

CPU times: user 14.5 s, sys: 57.6 ms, total: 14.6 s
Wall time: 14.6 s


14.573182344436646

In [5]:
# align the measurements captured at different wavelengths - could cause issue when using masks obtained
# at 550nm as the images are slightly shifted 
from processingmm.addons import align_wavelengths
directories = ['/media/elea/ssd/NPP_dev']

PDDN_mode = 'both'
run_all = False
align_wavelengths.align_wavelenghts(directories, PDDN_mode, run_all, [600])

Aligning wavelenght: 600nm


  0%|                                                   | 0/198 [00:00<?, ?it/s]

 
 >> Exported X3D .cod file as: /media/elea/ssd/NPP_dev/TU/550x600xIrregular/2023-08-02_T_HORAO-107-BF_FR_S6_1/raw_data/600nm/600_Intensite_aligned.cod
 
 >> Exported X3D .cod file as: /media/elea/ssd/NPP_dev/TU/550x600xIrregular/2023-08-02_T_HORAO-107-BF_FR_S6_1/raw_data/600nm/600_Bruit_aligned.cod


100%|█████████████████████████████████████████| 198/198 [00:16<00:00, 11.81it/s]


In [39]:
# add the calibration matrices (A.cod and W.cod) to the folder for processing
# needed by Christopher
from processingmm.addons import add_calibration
add_calibration.add_calibration(directories)

100%|█████████████████████████████████████████████| 2/2 [00:00<00:00, 15.59it/s]

No calibration files in 600nm
